## Extinction Corrected Effective Temperature
The LSST cadence includes a lot of "revisits", meaning the same object imaged 
in different bandpasses about 30 minutes apart. See https://survey-strategy.lsst.io/baseline/wfd.html
This is an opportunity to get a colour measurement of the object.
The pair of bandpasses are not just g+r but can be several combinations, see `REVISITS` below,
where the numbers index the LSST bands, so (0,2) means u+r bands.

Because the pair of bands vary, it is difficult to understand the meaning 
-- how is a u-r colour different from a g-r colour? Therefore Lasair would like to compute an
effective temperature from the two fluxes, to serve as a universal measure. If you want blue, filter out
the low temperatures, rather than having different criteria for different pairs of bandpasses.
To make this temperature even more useful, we make extinction correction first.

The executive summary:
- If g-r is 0.99, with no extinction, then the temperature is 3,620K.
- If g-r is 0.99 and very dusty, then the temperature is 7,740 K.

In [1]:
# The LSST bands, wavelength in microns
WL    = [0.380,     0.500,     0.620,     0.740,     0.880,     1.000, ]
BANDS = ['u',       'g',       'r'  ,     'i'  ,     'z'  ,     'y'    ]

In [2]:
# the revisit strategy
# u+g, u+r, g+r, r+i,  i+z,  z+y
# from https://survey-strategy.lsst.io/baseline/wfd.html
REVISITS = [(0,1), (0,2), (1,2), (2,3), (3,4), (4,5)]

These coefficients are for the extinction correction. 
For the g-filter, for example, an E(B-V) value of 0.5 
would increase the g magnitude number by 0.5*3.237 = 1.618 mags,
equivalently decrease the flux by a factor of 10^(1.618/2.5) = 4.43.

In [3]:
# Multiplier for EBV for magnitude correction
# Table 6: "F99 Reddening in Different Bandpasses" of Schlafly & Finkbeiner (2011) using RV = 3.1
EXTCOEF   = [4.145,    3.237,    2.273,    1.684,    1.323,    1.088]

You need to install the `dustmaps` package

In [4]:
import sys, math
sys.path.append('/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/')
from dustmaps.sfd import SFDQuery
from astropy.coordinates import SkyCoord

/Users/rwillia5/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


### Flux and magnitude
LSST is all flux, but people like colour as magnitude differences. Here are the converters.

In [5]:
def mag2flux(mag):
    # flux in nanoJ
    flux =  math.pow(10, (31.4 - mag)/2.5)
    return flux
def flux2mag(flux):
    # flux in nanoJ
    mag = 31.4 - 2.5*math.log10(flux)
    return mag

### Blackbody spectrum
This formula is "per Hertz" to match up with the definition of Jansky.
The number 'hck' is planck * speed of light / boltzmann, with temperature in kilokelvin and wavelength in microns.
There is also the derivative with respect to T so that we can do a Newton solver.
The commented out code shows that the derivative formula is numerically correct.

In [6]:
def blackbody(wl, T):
    hck = 14.387
    q = math.exp(hck/(wl*T))
    return math.pow(wl, -3.0) /(q - 1)

def dblackbody(wl, T):
    hck = 14.387
    q = math.exp(hck/(wl*T))
    return math.pow(wl, -3.0) * (hck/(wl*T*T)) * q/((q-1)*(q-1))
    
# show that the derivative formula is right
#T = 8
#wl = 0.7
#dT = 0.0001
#d1 = (blackbody(wl, T+dT) - blackbody(wl, T))/dT
#d2 = dblackbody(wl, T)
#print(d1, d2)

### Newton solver
Given a flux ratio `fluxrat` and two wavelengths `wl1` and `wl2`, we want to solve for the temperature `T`:
```
blackbody(wl1, T)/blackbody(wl2, T) = fluxrat

--> blackbody(wl1, T) - fluxrat* blackbody(wl2, T) = 0
```
However the flux ratio is first modified because of extinction, as the different 
wavelengths are extincted differently. See `EXTCOEF` above. First we make a magnitude change, 
then convert that to a multiplier on the flux ratio.

In [7]:
def solve_for_temperature(fluxrat, i1, i2, ebv=0):
    
    # modify fluxrat for extinction
    mag_corr = ebv*(EXTCOEF[i1] - EXTCOEF[i2])
    fluxrat_corr = fluxrat*math.pow(10, mag_corr/2.5)
    
    T = 12  # guess 12,000 kelvin
    wl1 = WL[i1]
    wl2 = WL[i2]
    for i in range(50):
        try:
            fr  =  blackbody(wl1, T) - fluxrat_corr* blackbody(wl2, T)
            dfr = dblackbody(wl1, T) - fluxrat_corr*dblackbody(wl2, T)
        except:
            return None
        dT = fr/dfr
        T = T - dT
        if abs(dT) < 0.001: return T # convergence
            
        if T < 0: return None  # not interested in negative temperatures
    return None

### Everything below is presentation

In [8]:
def printline(fluxrat, i1, i2, ebv):
    magdiff = -2.5*math.log10(fluxrat)
    print('%5.2f   %5.2f |' % (fluxrat, magdiff), end='')
    for (i1,i2) in REVISITS:
        s = solve_for_temperature(fluxrat, i1, i2, ebv)
        if s:
            print('%8.2f' % s, end='')
        else:
            print('     -- ', end='')
    print()

In [9]:
def make_table(ebv):
    print('RA,Dec = %.1f,%.1f, E(B-V) = %.3f' % (ra, decl, ebv))
    print('Temperature in kK for given flux ratio in given bands')
    print('Flux rat| m-m |', end='')
    for (i1,i2) in REVISITS:
        print('%s-%s     ' % (BANDS[i1], BANDS[i2]), end='')
    print()

    for i in range(10):
        fluxrat = 0.02*i + 0.01
        printline(fluxrat, i1, i2, ebv)
    for i in range(20):
        fluxrat = 0.1*i + 0.2
        printline(fluxrat, i1, i2, ebv)   

### Clear and dusty
First we make a run with a clear sky, far from milky way, E(B-V)=0.027.
The we make a run with a dusty sky, near the galactic centre, E(B-V)=0.919.

In [10]:
sfd = SFDQuery()

# clear view
ra   = 180
decl =  50
c = SkyCoord(ra, decl, unit="deg", frame='icrs')
ebv = float(sfd(c))
make_table(ebv)

print('================')

# dusty
ra   = 270
decl = -30
c = SkyCoord(ra, decl, unit="deg", frame='icrs')
ebv = float(sfd(c))
make_table(ebv)

RA,Dec = 180.0,50.0, E(B-V) = 0.027
Temperature in kK for given flux ratio in given bands
Flux rat| m-m |u-g     u-r     g-r     r-i     i-z     z-y     
 0.01    5.00 |    1.68    2.43    1.07    0.73    0.60    0.39
 0.03    3.81 |    2.11    2.97    1.35    0.94    0.77    0.51
 0.05    3.25 |    2.39    3.32    1.54    1.07    0.88    0.58
 0.07    2.89 |    2.63    3.59    1.70    1.18    0.98    0.65
 0.09    2.61 |    2.83    3.83    1.84    1.29    1.06    0.70
 0.11    2.40 |    3.02    4.04    1.97    1.38    1.14    0.76
 0.13    2.22 |    3.20    4.24    2.09    1.47    1.21    0.81
 0.15    2.06 |    3.37    4.42    2.21    1.56    1.28    0.86
 0.17    1.92 |    3.53    4.60    2.33    1.64    1.35    0.91
 0.19    1.80 |    3.69    4.76    2.44    1.73    1.42    0.96
 0.20    1.75 |    3.77    4.85    2.50    1.77    1.46    0.99
 0.30    1.31 |    4.54    5.61    3.05    2.19    1.80    1.24
 0.40    0.99 |    5.30    6.33    3.62    2.63    2.17    1.52
 0.50    0.75 